In [ ]:
!pip3 install -q -U pip
!apt-get install -y rustc
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl
!pip3 install -q -U datasets
!pip3 install -q -U transformers

SyntaxError: invalid syntax (ipython-input-3003098181.py, line 1)

In [ ]:

!pip3 install -q -U pip# Upgrade pip
!apt-get install -y rustc# Install Rust (needed for tokenizers if build is required)

!pip3 install -q -U \# Install compatible LLM fine-tuning stack (LoRA / PEFT / TRL)

  transformers==4.38.2 \ #For loading the model and tokenizer
  peft==0.8.2 \ #Efficient fine-tunig(LoRA)
  accelerate==0.27.1 \ #Multi-GPU/ mixed precision
  datasets==2.17.0 \ #Loading datasets
  bitsandbytes==0.42.0 \ #Loaddin 8bit and 4bits
  trl==0.7.10 #Training library


IndentationError: unexpected indent (ipython-input-2777100723.py, line 6)

In [ ]:
import os
import torch #For CUDA kernals/low-level speed
from datasets import load_dataset
from google.colab import userdata
import transformers
from trl import SFTTrainer  #For traning loop
from peft import LoraConfig   #For fine-tuning using LoRA
from transformers import AuotoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

ImportError: cannot import name 'AuotoTokenizer' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [ ]:
os.environ['HUGGINGFACE_TOKEN'] = userdata.get('HUGGINGFACE_TOKEN')

In [ ]:
model_id= "ggogle/gemma-2b"
#For fine-tuning trnsforming 32bit to 4bits using bnb config
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
  )

In [ ]:
tokenizer= AutoTokenizer.from_pretrained(model_id, token=os.environ['HUGGINGFACE_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             dvice_map={"":0}, #llm run on may devices and devicemao helo us to shoose where to run (if device_map = auto that's mean auto split layres nif =0 mean put everything on gpu 0)
                                            token=os.environ['HUGGINGFACE_TOKEN']
                                             )


In [ ]:
text="Quote: Imaginatons is more,"
device = "cuda:0"
inputs=tokneizer(text, return_tensts="pt").to(device)
outputs= model.generate(**inputs, max_new_tokens=20)
print(toknizer.decode(outputs[0], skip_special_tokens=True))
#

In [ ]:
lora_config = LoraConfig(
    r=8,
 atgre_modules = ["q_proj", "o_proj", "k_proj",""gate_proj", "up_proj",""down_proj","v_proj"],
    task_tyoe = "CAUSAL_LM"
)

In [ ]:
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data= data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [ ]:
data['train']['quote']

In [ ]:
def formatting_func(example):
  text = f"Quote: {example['quote'][0]}"\nAuthor: {example['author'][0]}"
  return text

In [ ]:
trainer = SFTTrainer(
    model=model,
    args =transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      warmup_steps=2,
      max_steps=100,
      learning_rate=2e-4,
      fp16=True,
      logging_steps=1,
      output_dir="output",
      optim="paged_adamw_8bit",
    ),
    peft_config = lora_config,
    formatting_func=formatting_func,

    )



In [ ]:
tranier.train()

In [ ]:
text="Quote: Imaginatons is more,"
device = "cuda:0"
inputs=tokneizer(text, return_tensts="pt").to(device)
outputs= model.generate(**inputs, max_new_tokens=20)
print(toknizer.decode(outputs[0], skip_special_tokens=True))
#